## Walktrap Community Detection

This code uses `CDlib - Community Discovery Library`
See the [link](https://cdlib.readthedocs.io/en/latest/) to check out documentation

In [137]:
import numpy as np
import itertools
import pandas as pd
import matplotlib.pyplot as plt

import networkx as nx
from cdlib import algorithms

# import data
df = pd.read_csv('../edge_files/edge_mean_th95.csv')

In [138]:
df = df.astype(int)
df = df[['dst','src']]
df

,dst,src
0,6,0
1,94,0
2,136,0
3,2,1
4,59,1
...,...,...
333,123,137
334,127,137
335,17,139
336,106,139


In [139]:
# define edge_list (u,v)
edge_list = list(df.itertuples(index=False, name=None))
len(edge_list)

338

In [140]:
# define node_list
node_list = [x for x in range(max(df['dst']))]
len(node_list)

141

In [141]:
from igraph import *

# Create Graph 
g = Graph()
print(g) # IGRAPH U--- 0 0 --

IGRAPH U--- 0 0 --


In [142]:
node_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140]

In [143]:
# vertex 추가
g.add_vertices(142)

print(g) # IGRAPH U--- 141 0 --

IGRAPH U--- 142 0 --


In [144]:
edge_list

[(6, 0),
 (94, 0),
 (136, 0),
 (2, 1),
 (59, 1),
 (85, 1),
 (97, 1),
 (1, 2),
 (34, 2),
 (19, 3),
 (89, 3),
 (15, 4),
 (114, 4),
 (121, 4),
 (28, 5),
 (62, 5),
 (86, 5),
 (111, 5),
 (123, 5),
 (127, 5),
 (137, 5),
 (0, 6),
 (28, 6),
 (30, 6),
 (32, 6),
 (48, 6),
 (86, 6),
 (110, 6),
 (119, 6),
 (141, 6),
 (38, 7),
 (70, 7),
 (30, 8),
 (119, 8),
 (26, 9),
 (91, 9),
 (11, 10),
 (23, 10),
 (10, 11),
 (134, 11),
 (70, 13),
 (98, 13),
 (4, 15),
 (118, 17),
 (139, 17),
 (3, 19),
 (129, 20),
 (33, 22),
 (10, 23),
 (9, 26),
 (50, 26),
 (91, 26),
 (132, 26),
 (5, 28),
 (6, 28),
 (32, 28),
 (50, 28),
 (62, 28),
 (69, 28),
 (86, 28),
 (100, 28),
 (110, 28),
 (119, 28),
 (123, 28),
 (127, 28),
 (136, 28),
 (137, 28),
 (86, 29),
 (100, 29),
 (126, 29),
 (130, 29),
 (6, 30),
 (8, 30),
 (36, 30),
 (75, 30),
 (110, 30),
 (119, 30),
 (125, 30),
 (6, 32),
 (28, 32),
 (43, 32),
 (50, 32),
 (100, 32),
 (137, 32),
 (22, 33),
 (91, 33),
 (126, 33),
 (127, 33),
 (130, 33),
 (2, 34),
 (42, 34),
 (135, 35),
 (

In [145]:
# g.add_edges([(6, 0)])
# print(g)

In [146]:
# edge 추가
for edge in edge_list:
    print(f'adding edge {edge}')
    g.add_edges([edge])

print(g)

adding edge (6, 0)
adding edge (94, 0)
adding edge (136, 0)
adding edge (2, 1)
adding edge (59, 1)
adding edge (85, 1)
adding edge (97, 1)
adding edge (1, 2)
adding edge (34, 2)
adding edge (19, 3)
adding edge (89, 3)
adding edge (15, 4)
adding edge (114, 4)
adding edge (121, 4)
adding edge (28, 5)
adding edge (62, 5)
adding edge (86, 5)
adding edge (111, 5)
adding edge (123, 5)
adding edge (127, 5)
adding edge (137, 5)
adding edge (0, 6)
adding edge (28, 6)
adding edge (30, 6)
adding edge (32, 6)
adding edge (48, 6)
adding edge (86, 6)
adding edge (110, 6)
adding edge (119, 6)
adding edge (141, 6)
adding edge (38, 7)
adding edge (70, 7)
adding edge (30, 8)
adding edge (119, 8)
adding edge (26, 9)
adding edge (91, 9)
adding edge (11, 10)
adding edge (23, 10)
adding edge (10, 11)
adding edge (134, 11)
adding edge (70, 13)
adding edge (98, 13)
adding edge (4, 15)
adding edge (118, 17)
adding edge (139, 17)
adding edge (3, 19)
adding edge (129, 20)
adding edge (33, 22)
adding edge (10, 23

In [147]:
# Create Graph 
G = nx.Graph()

# Add node and edge to graph
G.add_nodes_from(node_list)
G.add_edges_from(edge_list)

# Delete node which does not have neighborhood
print('Delete node with no neighborhood: {} \n'.format(list(nx.isolates(G))))
G.remove_nodes_from(list(nx.isolates(G)))
print('Number of node:',len(G.nodes))

Delete node with no neighborhood: [12, 14, 16, 18, 21, 24, 25, 27, 31, 44, 47, 49, 51, 52, 53, 57, 60, 61, 63, 64, 65, 71, 72, 73, 74, 77, 79, 80, 81, 84, 87, 88, 92, 93, 99, 101, 103, 104, 105, 107, 109, 112, 113, 116, 117, 124, 128, 131, 133, 138, 140] 

Number of node: 91


In [152]:
import igraph as ig

In [154]:
coms = algorithms.walktrap(g)

ModuleNotFoundError: Optional dependency not satisfied: install igraph to use the selected feature.

In [150]:
# criterion for most valuable edge (can be changed)
def most_central_edge(G):
    centrality = betweenness(G, weight="weight")
    # print(centrality)
    return max(centrality, key=centrality.get)

In [151]:
# community detection with betweenness
communities = girvan_newman(G, most_valuable_edge=most_central_edge)

node_groups = []
for com in next(communities):
    node_groups.append(list(com))


plt.figure(figsize=(10,10))
pos = nx.spring_layout(G, scale=4)
nx.draw(G, pos, node_color=range(len(G.nodes)), node_size=500, with_labels=True, cmap=plt.cm.Blues)
plt.title('Community Detection with betweenness')
plt.show()
plt.savefig('betweenness.jpg')

NameError: name 'girvan_newman' is not defined